# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846824180876                   -0.70    4.5
  2   -7.852321776866       -2.26       -1.53    1.0
  3   -7.852646004122       -3.49       -2.52    3.2
  4   -7.852646677169       -6.17       -3.35    2.5
  5   -7.852646685908       -8.06       -4.76    1.2
  6   -7.852646686726       -9.09       -5.25    4.0
  7   -7.852646686730      -11.46       -5.98    2.0
  8   -7.852646686730      -12.62       -7.59    2.0
  9   -7.852646686730   +  -14.75       -7.98    3.0
 10   -7.852646686730      -15.05       -9.02    2.5
 11   -7.852646686730   +  -14.57      -10.02    1.5
 12   -7.852646686730      -14.27      -10.68    3.5
 13   -7.852646686730   +  -14.75      -12.04    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846808767076                   -0.70           4.8
  2   -7.852525116588       -2.24       -1.63   0.80    2.0
  3   -7.852636418326       -3.95       -2.71   0.80    1.0
  4   -7.852646521671       -5.00       -3.30   0.80    2.2
  5   -7.852646682671       -6.79       -4.17   0.80    1.2
  6   -7.852646686437       -8.42       -4.88   0.80    1.5
  7   -7.852646686725       -9.54       -5.81   0.80    2.0
  8   -7.852646686730      -11.34       -6.74   0.80    2.0
  9   -7.852646686730      -12.76       -7.44   0.80    2.5
 10   -7.852646686730      -14.75       -8.92   0.80    1.5
 11   -7.852646686730      -14.57       -8.53   0.80    3.2
 12   -7.852646686730   +  -14.75       -8.95   0.80    1.8
 13   -7.852646686730   +  -15.05       -9.87   0.80    1.0
 14   -7.852646686730   +  -15.05      -10.93   0.80    1.2
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.449603e+01     4.243853e+00
 * time: 0.5199978351593018
     1     1.625171e+00     1.762913e+00
 * time: 0.7941279411315918
     2    -1.925300e+00     1.912386e+00
 * time: 0.8231749534606934
     3    -3.734570e+00     1.686054e+00
 * time: 0.8654317855834961
     4    -5.231406e+00     1.503725e+00
 * time: 0.906721830368042
     5    -6.618231e+00     9.446210e-01
 * time: 0.9547607898712158
     6    -7.227718e+00     5.452916e-01
 * time: 0.9977920055389404
     7    -7.502033e+00     4.120428e-01
 * time: 1.0408217906951904
     8    -7.638555e+00     1.507858e-01
 * time: 1.0790138244628906
     9    -7.726615e+00     1.643631e-01
 * time: 1.108285903930664
    10    -7.769841e+00     1.173319e-01
 * time: 1.1380939483642578
    11    -7.797787e+00     1.012401e-01
 * time: 1.167923927307129
    12    -7.827237e+00     5.917782e-02
 * time: 1.1973178386688232
    13    -7.843578e+00     2.816654e-02
 * time: 1.2273509502410

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846863435525                   -0.70    4.5
  2   -7.852316340985       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686712                   -2.53
  2   -7.852646686730      -10.76       -6.03
  3   -7.852646686730      -15.05      -12.67


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1730912787263513e-13
|ρ_newton - ρ_scfv| = 2.289463869389103e-13
|ρ_newton - ρ_dm|   = 1.3060480679730836e-9
